# simulation with ivs data

### Imports
Import the required libraries

In [1]:
# package(s) related to time, space and id
import datetime, time
import platform
import random
import os
import pathlib

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy

# spatial libraries 
import pyproj
import shapely.geometry
from shapely.geometry import Point
import shapely
import geopandas as gpd
import movingpandas as mpd

# package(s) for data handling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pickle

# OpenTNSIM
import opentnsim
import opentnsim.core as core
import opentnsim.graph_module as graph_module
import opentnsim.plot as plot


# dtv_backend
import dtv_backend.fis as fis
import dtv_backend.network
import dtv_backend.network.network_utilities
import dtv_backend.postprocessing
import dtv_backend.simple
import dtv_backend.simulate

# Used for mathematical functions
import math             
import numpy as np

# Used for making the graph to visualize our problem
import networkx as nx  
import requests
import io

### Create graph

In [2]:
#Load lock info
url = "https://zenodo.org/records/6673604/files/FIS_locks_grouped.geojson?download=1"
resp = requests.get(url)
stream = io.BytesIO(resp.content)
locks_gdf = gpd.read_file(stream)

In [3]:
#Load graph
url = "https://zenodo.org/record/6673604/files/network_digital_twin_v0.3.pickle?download=1"
graph1 = dtv_backend.fis.load_fis_network(url).copy()

# #remove edges without classification
# for edge in graph1.edges:
#     if str(graph1.edges[edge]['Classification']) == 'nan':
#         graph1.remove_edge(edge[0], edge[1])

graph = graph_module.Graph()
graph.graph = graph1
graph.graph_info = opentnsim.utils.info(graph.graph)

In [4]:
cemt_classes_ordered = {
            "0": 0,
            "I": 1,
            "II": 2,
            "III": 3,
            "IV": 4,
            "IVa": 5,
            "Va": 6,
            "Vb": 7,
            "VIa": 8,
            "VIb": 9,
            "VIc": 10,
            "VIIa": 11,
}
# define synonyms:
code_synonyms = {
        "_0": "0",
        "V_A": "Va",
        "V_B": "Vb",
        "VI_A": "VIa",
        "VI_B": "VIb",
        "VI_C": "VIc",
    }

for edge in graph.graph.edges:
    # replace synonyms
    if graph.graph.edges[edge]["Code"] in code_synonyms:
        graph.graph.edges[edge]["Code"] = code_synonyms[graph.graph.edges[edge]["Code"]]


        

In [6]:
# save graph in geopanda
df_edges = nx.to_pandas_edgelist(graph)
gpd.GeoDataFrame(df_edges).to_file('plots_routes/edges_zonder_afzetting')


AttributeError: 'Graph' object has no attribute 'edges'

### Make vessels and paths

##### read data

In [7]:
if os.path.isfile('data/ivs/ivs-2024-geocoded_filtered.pkl'):
    data = pickle.load(open('data/ivs/ivs-2024-geocoded_filtered.pkl', 'rb'))
else:
    # # lees data in.
    data = gpd.read_file("data\ivs\ivs-2024-geocoded.gpkg")

    # #filter data op bestaande iso datum en geometry.
    data['datetime'] = pd.to_datetime(data['v05_06_begindt_evenement_iso'], format = 'ISO8601', errors = 'coerce')
    data.dropna(subset = ['datetime', 'geometry'], inplace = True)


filter only on rotterdam to duisburg, 2023. Only keep ships in january 2023

In [8]:
t_begin = pd.Timestamp('2023-10-10', tz='UTC')
t_end = pd.Timestamp('2023-10-12', tz='UTC')

condition_1 = (data.UNLO_bestemming.str.startswith("DE")) | (data.UNLO_herkomst.str.startswith("DE"))
condition_2 = data.datetime>=(t_begin)
condition_3 = data.datetime<(t_end)
condition_4 = ~ (data['SK_CODE'].isna()) & data['SK_CODE'].str.startswith("M") #alleen schepen met code M
condition_5 =  data.UNLO_bestemming != data.UNLO_herkomst
idx = np.logical_and.reduce([
 #condition_1,
 condition_2,
 condition_3,
 condition_4,
 condition_5
])
data = data[idx]
#data.to_pickle('data/ivs/ivs-2024-geocoded_filtered.pkl')
len(data)

3475

##### Create vessels

In [9]:
from networkx.exception import NetworkXNoPath
from dtv_backend.fis import path_restricted_to_rws_class
import json
from dtv_backend.lock import AVERAGE_SHIP_LENGTH
from tqdm import tqdm

# Make a class out of mix-ins
TransportResource = type('TransportResource', 
                         (core.Identifiable, core.ContainerDependentMovable, 
                          core.HasResource, core.Routable,
                          core.VesselProperties,
                         core.ExtraMetadata), 
                         {})
#define speed: 
def compute_v_provider(v_empty, v_full):
    return lambda x: 1

# define weight
def compute_weight(origin, target, dictionary):
    if dictionary['Code'] in ('IV', 'III', 'II', "I"): #niet langs te kleine wegen
        return None
    else:
        return dictionary['length_m']
    
# load saved routes
with open('data/ivs/vessel_routes_without_afzetting.json', 'r') as f:
    saved_routes = json.load(f)


vessels = []
failed_vessels = []
for index, row in tqdm(data.iterrows()):
    #determine path
    try:
        if str(row.name) in saved_routes.keys():
            path = saved_routes[str(row.name)]
        else:
            point_1 = fis.find_closest_node(graph.graph, Point(row.geometry.coords[0]))
            point_2 = fis.find_closest_node(graph.graph, Point(row.geometry.coords[-1]))
            path = path_restricted_to_rws_class(graph = graph.graph, 
                                            origin = point_1[0], 
                                            destination=point_2[0], 
                                            ship_rws_classe=f"M{row['v15_1_Scheepstype_RWS']}", 
                                            ordered_cemt_classes=cemt_classes_ordered) 
        #path = nx.dijkstra_path(graph.graph, point_1[0], point_2[0], weight=compute_weight)
        #determine capacity
        capacity = max(row.v18_Laadvermogen*1000, row.v38_Vervoerd_gewicht, 1)
        data_vessel = {"env": None,
                "name": row.name,
                "type": row['v15_1_Scheepstype_RWS'],
                "B": 8,
                "L": AVERAGE_SHIP_LENGTH,
                "route": path,
                "geometry": Point(row.geometry.coords[0]),  # lon, lat
                "capacity": capacity,
                "v": 0.5144*8, # 8 knopen
                "compute_v": compute_v_provider(v_empty=0.5144*8, v_full=0.5144*8),
                "departure_time": pd.to_datetime(row['v05_06_begindt_evenement_iso']),
                }
        vessel = TransportResource(**data_vessel)
        vessels.append(vessel)
    except NetworkXNoPath:
        failed_vessels.append(row.name)
    except ValueError:
        failed_vessels.append(row.name)
print(f"Failed vessels: {failed_vessels}")
        

# korte_route = nx.dijkstra_path(graph.graph, "8865735", "8861687")
# vessels = [TransportResource(**{
#     "env" : None,
#     "name": 'korte_route', 
#     "type": 'M6',
#     "B": 1, 
#     "L": 10,
#     "route": korte_route,
#     "geometry": Point(row.geometry.coords[0]),  # lon, lat
#     "capacity": capacity,
#     "v": 1,
#     "compute_v": compute_v_provider(v_empty=1, v_full=1),
# })]

3475it [09:16,  6.25it/s]

Failed vessels: [318, 2595, 2596, 4828, 7987, 7988, 10318, 11747, 12565, 12567, 13991, 14870, 16301, 16302, 18640, 19514, 23257, 23258, 24172, 27947, 27948, 27950, 28807, 31071, 31072, 32504, 32506, 33387, 33388, 33391, 34855, 34857, 37161, 38083, 39497, 39498, 39500, 40358, 42700, 45064, 47406, 47408, 47409, 48917, 52113, 52116, 53639, 53640, 53641, 54507, 58268, 61454, 63823, 63828, 65271, 73254, 77018, 79304, 79306, 80178, 81665, 81666, 81667, 86298, 86299, 88504, 88505, 90814, 95458, 97754, 100092, 100093, 102311, 102312, 102314, 103169, 104630, 109210, 109212, 109213, 110055, 110059, 112394, 121700, 121701, 123907, 129992, 133245, 133246, 134740, 134741, 137139, 137140, 139401, 140246, 142529, 143964, 146229, 146230, 146231, 146232, 147105, 148549, 148550, 148551, 149379, 149385, 150888, 150889, 151757, 153208, 153209, 153210, 154072, 155482, 155483, 155484, 155487, 156342, 157834, 157838, 157839, 160999, 163227, 164776, 165617, 165618, 165621, 167065, 167910, 169443, 170304, 1717

In [10]:
a = {vessel.name: vessel.route for vessel in vessels}
json.dump(a, open('data/ivs/vessel_routes_without_afzetting.json', 'w'))

# sluis maken


### Start simulation

In [11]:
def start(env, vessel):
    while True:
        #wait untill ship will start sailing
        time_departure = time.mktime(vessel.metadata['departure_time'].timetuple())
        yield env.timeout(time_departure-env.now)

        # start sailing
        vessel.log_entry_v0("Start sailing", env.now, "", vessel.geometry)
        yield from vessel.move()
        vessel.log_entry_v0("Stop sailing", env.now, "", vessel.geometry)
        
        if vessel.geometry == nx.get_node_attributes(env.FG, "geometry")[vessel.route[-1]]:
            break

In [12]:
# Start simpy environment
simulation_start = min([vessel.metadata['departure_time'] for vessel in vessels])

env = simpy.Environment(initial_time = time.mktime(simulation_start.timetuple()))
env.epoch = time.mktime(simulation_start.timetuple())


# Add graph to environment
#graph.add_resources(list(graph.graph.edges), np.ones(len(list(graph.graph.edges))), env)
env.FG = graph.graph



In [13]:
from dtv_backend.lock import Lock, Locks

locks = Locks(env)


In [14]:
import functools
for i, vessel in enumerate(vessels):
    # Add environment and path to the vessel
    vessel.env = env

    # add passing of a lock
    filled_pass_lock = functools.partial(locks.pass_lock_v2, vessel=vessel)
    vessel.on_pass_edge_functions = [filled_pass_lock]

    # Add the movements of the vessel to the simulation
    env.process(start(env, vessel))

env.run(until=env.timeout(60*60*24*2))

KeyboardInterrupt: 

### Obtain vessel log information
The cel below uses the vessel log. The core function *log_entry* is used, which takes four arguments:

- **Log.** A text to describe what is logged.
- **t.** The timestamp.
- **Value.**  The value for the log (for sailing this is the distance).
- **Geometry** The location of the vessel while loggin.

In [20]:
# save the logs and trajectories of vessels
log = gpd.GeoDataFrame()
for vessel in vessels:
    vessel_log = gpd.GeoDataFrame(vessel.logbook, geometry='Geometry')
    vessel_log['trajectory_id'] = f'vessel_{vessel.name}_trip_1'
    vessel_log['object_id'] = f'vessel_{vessel.name}'
    log = pd.concat([log, vessel_log])
mpd_log = mpd.TrajectoryCollection(log, traj_id_col='trajectory_id', obj_id_col='object_id', t='Timestamp')
mpd_log.to_line_gdf().to_file('plots_routes/trajectories_without_afzetting.gpkg')

d:\Users\ALPF\AppData\Local\pypoetry\Cache\virtualenvs\dtv-backend-pvs5eNot-py3.10\lib\site-packages\movingpandas\trajectory.py:166: MissingCRSWarning:

Trajectory generated without CRS. Computations will use Euclidean distances.

d:\Users\ALPF\AppData\Local\pypoetry\Cache\virtualenvs\dtv-backend-pvs5eNot-py3.10\lib\site-packages\movingpandas\trajectory.py:166: MissingCRSWarning:

Trajectory generated without CRS. Computations will use Euclidean distances.

d:\Users\ALPF\AppData\Local\pypoetry\Cache\virtualenvs\dtv-backend-pvs5eNot-py3.10\lib\site-packages\movingpandas\trajectory.py:166: MissingCRSWarning:

Trajectory generated without CRS. Computations will use Euclidean distances.

d:\Users\ALPF\AppData\Local\pypoetry\Cache\virtualenvs\dtv-backend-pvs5eNot-py3.10\lib\site-packages\movingpandas\trajectory.py:166: MissingCRSWarning:

Trajectory generated without CRS. Computations will use Euclidean distances.

d:\Users\ALPF\AppData\Local\pypoetry\Cache\virtualenvs\dtv-backend-pvs5eNot-

In [21]:
# Save the logs of the locks
lock_dfs = []
for id, lock_object in locks.locks_resources.items():
    lock = pd.DataFrame(lock_object.logbook)
    lock_properties = pd.DataFrame(lock["Value"].values.tolist())
    lock_df = pd.concat([lock, lock_properties], axis=1)
    lock_df['lock_id'] = id
    lock_dfs.append(lock_df)

all_locks_df = pd.concat(lock_dfs, axis=0)
all_locks_df = all_locks_df.drop(columns=['Value'])
all_locks_df.to_csv('plots_routes/locks_without_afzetting_.csv')

### Visualization of path
If you get an error regarding ffmpeg use [this](https://stackoverflow.com/questions/13316397/matplotlib-animation-no-moviewriters-available) answer. You have to install ffmpeg in your Conda environment. It can be done using the following command.

```bash
#Install ffmpeg using Conda
conda install -c conda-forge ffmpeg
```

In [22]:
AVERAGE_SHIP_LENGTH

50

In [ ]:
# visualise vessel movements based on the information included in the vessel.log
plot.vessel_kml(env, vessels, stepsize = 60)
#plot.graph_kml(env)


In [ ]:
! start explorer .